In [1]:
%matplotlib inline

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nose.tools import *

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, RANSACRegressor

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler

In [5]:
housing = pd.read_fwf('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data', header=None)

In [6]:
housing

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [8]:
housing.columns = ["crime_ rate", "zoned_ land", "industry ", "bounds_ river",
                   "nox_ conc", " rooms", " age", " distance", " highways", " tax", "pt_ ratio",
                   "b_ estimator", "pop_ status", "price"
                  ]

In [10]:
housing.describe().T

,count,mean,std,min,25%,50%,75%,max
crime_ rate,506.0,1.716290,2.653510,0.00632,0.0819,0.250895,2.326718,9.96654
zoned_ land,506.0,11.363636,23.322453,0.00000,0.0000,0.000000,12.500000,100.00000
industry,506.0,11.136779,6.860353,0.46000,5.1900,9.690000,18.100000,27.74000
bounds_ river,506.0,0.069170,0.253994,0.00000,0.0000,0.000000,0.000000,1.00000
nox_ conc,506.0,0.554695,0.115878,0.38500,0.4490,0.538000,0.624000,0.87100
rooms,506.0,6.284634,0.702617,3.56100,5.8855,6.208500,6.623500,8.78000
age,506.0,68.574901,28.148861,2.90000,45.0250,77.500000,94.075000,100.00000
distance,506.0,3.696228,1.999689,0.58570,2.0737,3.107300,5.112625,9.22290
highways,506.0,4.332016,1.417166,1.00000,4.0000,4.000000,5.000000,8.00000
tax,506.0,408.237154,168.537116,187.00000,279.0000,330.000000,666.000000,711.00000


In [12]:
housing.shape

(506, 14)

Тук трябва да има дълъг процес по изследване на данните още преди да започнем да ги обработваме. Има ли нулеви стойности, Какво е съдържанието на колоните. Цените например какво е отношението при тях. Има ли корелации между променливите и какви, линейни ли са връзките м/у променливите ит.н. За да обосновем защо ще използваме линейна регресия при моделирането.

In [16]:
housing_prices = housing.price

In [17]:
housing_attributes = housing.drop('price', axis=1)

In [18]:
housing_attributes.shape, housing_prices.shape

((506, 13), (506,))

### Скалираме данните

In [20]:
scaler = MinMaxScaler()
scaler.fit(housing_attributes)

MinMaxScaler()

In [22]:
housing_attributes_scaled = scaler.transform(housing_attributes)

In [38]:
housing_attributes.head(5)

,crime_ rate,zoned_ land,industry,bounds_ river,nox_ conc,rooms,age,distance,highways,tax,pt_ ratio,b_ estimator,pop_ status
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33


In [24]:
# проверяваме минималната стойност по колони
housing_attributes_scaled.min(axis=0)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [26]:
# проверяваме максималната стойност по колони
housing_attributes_scaled.max(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [27]:
# проверяваме средната стойност по колони
housing_attributes_scaled.mean(axis=0)

array([0.17167998, 0.11363636, 0.39137752, 0.06916996, 0.34916679,
       0.52186901, 0.67636355, 0.36013158, 0.47600226, 0.42220831,
       0.62292911, 0.89856783, 0.30140903])

### Създаваме модела, който да обучим с наличните ни данни

In [29]:
linear_regression = LinearRegression()
linear_regression.fit(housing_attributes_scaled, housing_prices)

LinearRegression()

In [30]:
linear_regression.coef_

array([  2.08448854,   1.49403979,   0.34690497,   3.00565375,
        -7.54441381,  22.43940145,   0.27658754,  -9.35981793,
         1.35281035,  -1.26826011,  -9.07603108,   3.74177288,
       -19.03479847])

In [31]:
linear_regression.intercept_

23.689291534676947

In [35]:
# взимаме 10 записа от скалираните данни. ако е само един запис го правим на вектор стълб с ressape(-1,1)
housing_attributes_scaled[0:10] 

array([[0.        , 0.18      , 0.06781525, 0.        , 0.31481481,
        0.57750527, 0.64160659, 0.40572176, 0.        , 0.20801527,
        0.28723404, 1.        , 0.08967991],
       [0.00210738, 0.        , 0.24230205, 0.        , 0.17283951,
        0.5479977 , 0.78269825, 0.50727087, 0.14285714, 0.10496183,
        0.55319149, 1.        , 0.2044702 ],
       [0.00210538, 0.        , 0.24230205, 0.        , 0.17283951,
        0.6943859 , 0.59938208, 0.50727087, 0.14285714, 0.10496183,
        0.55319149, 0.98973725, 0.06346578],
       [0.0026154 , 0.        , 0.06304985, 0.        , 0.15020576,
        0.65855528, 0.44181256, 0.63405965, 0.28571429, 0.06679389,
        0.64893617, 0.99427606, 0.03338852],
       [0.00629805, 0.        , 0.06304985, 0.        , 0.15020576,
        0.68710481, 0.52832132, 0.63405965, 0.28571429, 0.06679389,
        0.64893617, 1.        , 0.09933775],
       [0.0023624 , 0.        , 0.06304985, 0.        , 0.15020576,
        0.54972217, 0.57466

Подаваме тези 10 записа на обучения вече модел, за да видим резултатите, като ги сверяваме с нашия таргет/изх.променлива/у

In [34]:
linear_regression.predict(housing_attributes_scaled[0:10])

array([30.10947333, 25.12810976, 31.00785588, 29.04535626, 28.48368175,
       25.44231142, 23.23025545, 20.07399474, 11.87137324, 19.61327434])

In [36]:
prices[0:10]

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
5    28.7
6    22.9
7    27.1
8    16.5
9    18.9
Name: price, dtype: float64

In [39]:
prices[0:10].values

array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9])

In [40]:
housing_prices[0:10].values

array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9])

Може и със случайна извадка да направим същото като скалираните данни връщаме в дейтафрейм

In [44]:
random_sample = pd.DataFrame(housing_attributes_scaled).sample(10)

In [45]:
random_sample

,0,1,2,3,4,5,6,7,8,9,10,11,12
389,0.817795,0.000,0.646628,0.0,0.648148,0.350450,0.988671,0.132265,0.428571,0.914122,0.808511,1.000000,0.527594
353,0.001081,0.900,0.057185,0.0,0.051440,0.606821,0.341916,0.178391,0.571429,0.000000,0.468085,0.968632,0.076435
160,0.127220,0.000,0.700880,1.0,0.452675,0.515233,0.923790,0.140404,0.571429,0.412214,0.223404,0.853800,0.104029
404,0.152896,0.000,0.646628,0.0,0.633745,0.377467,0.849640,0.118291,0.428571,0.914122,0.808511,0.829946,0.707781
357,0.385873,0.000,0.646628,1.0,0.792181,0.543016,0.907312,0.222236,0.428571,0.914122,0.808511,0.985980,0.318433
286,0.001338,0.800,0.047654,0.0,0.000000,0.511401,0.294542,0.984520,0.000000,0.103053,0.595745,0.860558,0.309051
457,0.822699,0.000,0.646628,0.0,0.674897,0.455068,0.797116,0.253960,0.428571,0.914122,0.808511,0.008019,0.419702
11,0.011159,0.125,0.271628,0.0,0.286008,0.469055,0.823893,0.653105,0.571429,0.236641,0.276596,1.000000,0.318433
229,0.043720,0.000,0.210411,0.0,0.244856,0.573098,0.190525,0.322952,1.000000,0.229008,0.510638,0.958243,0.056015
120,0.006292,0.000,0.923387,0.0,0.403292,0.442422,0.687951,0.193581,0.142857,0.001908,0.691489,0.980458,0.348786


In [46]:
linear_regression.predict(random_sample)

array([13.40926268, 35.4036552 , 33.72814848,  8.7643957 , 21.81325626,
       19.0455002 , 12.71514783, 22.11901895, 30.8431165 , 20.23064941])

In [49]:
housing_prices.loc[random_sample.index].values

array([11.5, 30.1, 27. ,  8.5, 21.7, 20.1, 13.5, 18.9, 31.5, 22. ])

In [58]:
linear_regression.score(housing_attributes_scaled, housing_prices)

0.7198065414937174

In [53]:
ransac = RANSACRegressor()

In [54]:
ransac.fit(housing_attributes_scaled, housing_prices)

RANSACRegressor()

In [55]:
ransac.estimator_.coef_

array([  5.53099151,   0.71289054,  -2.80160002,   0.43322147,
        -0.85480161,  20.76967452,  -0.06189361, -11.14806505,
        -1.21092268,  -4.96259202,  -9.96162749,   3.45156472,
       -19.05135435])

In [56]:
ransac.estimator_.intercept_

27.571833326297607

In [57]:
ransac.score(housing_attributes_scaled, housing_prices)

0.6866008258637762

In [59]:
ransac.inlier_mask_

array([False, False,  True,  True, False,  True,  True, False, False,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False,  True,  True, False,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False, False,  True,  True, False,  True,  True,  True,  True,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False, False, False,
       False, False,  True,  True, False,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [62]:
inliers = housing_attributes_scaled[ransac.inlier_mask_]
outliers = housing_attributes_scaled[~ransac.inlier_mask_]

In [64]:
ransac.score(inliers, housing_prices.loc[ransac.inlier_mask_])

0.9219398710484017

In [66]:
ransac.score(outliers, housing_prices.loc[~ransac.inlier_mask_])

0.5793736087379613

In [67]:
polynomials_features = PolynomialFeatures()

In [69]:
housing_attributes_scaled_poly = polynomials_features.fit_transform(housing_attributes_scaled)

In [72]:
quadratic_regretion = LinearRegression()
quadratic_regretion.fit(housing_attributes_scaled_poly, housing_prices)

LinearRegression()

In [73]:
quadratic_regretion.score(housing_attributes_scaled_poly, housing_prices)

0.8527926535083148

In [78]:
iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
iris.columns = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

In [79]:
iris

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [84]:
iris_attributes = iris.drop('class', axis=1)
iris_classes = iris['class']

In [85]:
iris_attributes_scaled = MinMaxScaler().fit_transform(iris_atributes)

In [86]:
logistic_regression = LogisticRegression(C=1e9)

In [88]:
logistic_regression.fit(iris_attributes_scaled, iris_classes)

LogisticRegression(C=1000000000.0)

In [89]:
logistic_regression.coef_

array([[-18.29679129,  29.19813694, -37.89417388, -38.3219437 ],
       [ 13.58611302,  -6.58018128,  -8.87560643,  -2.78717031],
       [  4.71067827, -22.61795566,  46.76978031,  41.10911401]])

In [90]:
logistic_regression.intercept_

array([ 25.9014194 ,  14.7266206 , -40.62803999])

In [91]:
logistic_regression.score(iris_attributes_scaled, iris_classes)

0.9866666666666667

In [92]:
PolynomialFeatures(degree=7).fit_transform(iris_attributes_scaled)

array([[1.00000000e+00, 2.22222222e-01, 6.25000000e-01, ...,
        5.77244430e-10, 3.54764806e-10, 2.18032537e-10],
       [1.00000000e+00, 1.66666667e-01, 4.16666667e-01, ...,
        5.77244430e-10, 3.54764806e-10, 2.18032537e-10],
       [1.00000000e+00, 1.11111111e-01, 5.00000000e-01, ...,
        3.24699992e-10, 2.66073604e-10, 2.18032537e-10],
       ...,
       [1.00000000e+00, 6.11111111e-01, 4.16666667e-01, ...,
        1.57581908e-01, 1.75247340e-01, 1.94893123e-01],
       [1.00000000e+00, 5.27777778e-01, 5.83333333e-01, ...,
        3.59963549e-01, 4.42455196e-01, 5.43851178e-01],
       [1.00000000e+00, 4.44444444e-01, 4.16666667e-01, ...,
        8.61097024e-02, 8.77723898e-02, 8.94671819e-02]])

In [98]:
pd.read_json('quotes.json')

,text,author,tags
0,“The world as we have created it is a process ...,Albert Einstein,"[change, deep-thoughts, thinking, world]"
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,"[abilities, choices]"
2,“There are only two ways to live your life. On...,Albert Einstein,"[inspirational, life, live, miracle, miracles]"
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,"[aliteracy, books, classic, humor]"
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,"[be-yourself, inspirational]"
5,“Try not to become a man of success. Rather be...,Albert Einstein,"[adulthood, success, value]"
6,“It is better to be hated for what you are tha...,André Gide,"[life, love]"
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison,"[edison, failure, inspirational, paraphrased]"
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt,[misattributed-eleanor-roosevelt]
9,"“A day without sunshine is like, you know, nig...",Steve Martin,"[humor, obvious, simile]"


In [109]:
imot = pd.read_json('imot.json')

In [110]:
imot.shape

(161, 3)

In [115]:
imot = imot.dropna()

In [116]:
imot.shape

(41, 3)

In [119]:
imot

,type_of_im,address,price
0,Продава 1-СТАЕН,"град Пловдив, Въстанически",29 500 EUR
6,Продава 1-СТАЕН,"град Пловдив, Въстанически",29 500 EUR
9,Продава 1-СТАЕН,"град Пловдив, Каменица 2",38 000 EUR
12,Продава 1-СТАЕН,"град Пловдив, Кършияка",23 310 EUR
15,Продава 1-СТАЕН,"град Пловдив, Кършияка",28 199 EUR
18,Продава 1-СТАЕН,"град Пловдив, Кючук Париж",20 213 EUR
21,Продава 1-СТАЕН,"град Пловдив, Кючук Париж",21 126 EUR
24,Продава 1-СТАЕН,"град Пловдив, Кючук Париж",21 199 EUR
27,Продава 1-СТАЕН,"град Пловдив, Кючук Париж",22 680 EUR
30,Продава 1-СТАЕН,"град Пловдив, Кючук Париж",24 300 EUR
